### 추천시스템 모델링

필요한 라이브러리 불러오기

In [2]:
import pandas as pd; import numpy as np
from sklearn.model_selection import train_test_split #데이터분할

#시각화
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from PIL import Image;from matplotlib import rc

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore') 

import matplotlib; import matplotlib.pyplot as plt
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

#모델링
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math

데이터 불러오기

In [12]:
#모델링에 사용할 데이터프레임 생성
df = pd.read_pickle('data/real_final_df_230115.pickle')
dfm = df.loc[:,['gender','height','weight','size','content','총장','어깨너비','가슴단면','소매길이','total_big','total_small',
                 'chongjang_big','chongjang_small','arm_big','arm_small','chest_big','chest_small','shoulder_big','shoulder_small']]
dfm.reset_index(drop=True, inplace=True)
dfm['gender'] = [1 if dfm['gender'][i]=='남성' else 0 for i in range(dfm.shape[0])]

#데이터 타입 변경
col = ['total_big','total_small','chongjang_big','chongjang_small','arm_big','arm_small',
       'chest_big','chest_small','shoulder_big','shoulder_small']
dfm[col] = dfm[col].apply(pd.to_numeric, errors = 'coerce')

In [6]:
#합치기
i=0
for i in range(dfm.shape[0]):
    if(dfm['total_big'][i]==1):
        dfm['chongjang_big'][i] = 1
        dfm['shoulder_big'][i] = 1
        dfm['chest_big'][i] = 1
        dfm['arm_big'][i] = 1
    else:
        pass
    if(dfm['total_small'][i]==1):
        dfm['chongjang_small'][i] = 1
        dfm['shoulder_small'][i] = 1
        dfm['chest_small'][i] = 1
        dfm['arm_small'][i] = 1
        
#둘다 1인거 뽑아내기
print('전체 : {}'.format(dfm[(dfm['total_big']==1)&(dfm['total_small']==1)].shape[0]))
print('어깨 : {}'.format(dfm[(dfm['shoulder_big']==1)&(dfm['shoulder_small']==1)].shape[0]))
print('가슴 : {}'.format(dfm[(dfm['chest_big']==1)&(dfm['chest_small']==1)].shape[0]))
print('소매 : {}'.format(dfm[(dfm['arm_big']==1)&(dfm['arm_small']==1)].shape[0]))
print('총장 : {}'.format(dfm[(dfm['chongjang_big']==1)&(dfm['chongjang_small']==1)].shape[0]))

shoulder  = (dfm['shoulder_big']==1)&(dfm['shoulder_small']==1)
chest     = (dfm['chest_big']==1)&(dfm['chest_small']==1)
arm       = (dfm['arm_big']==1)&(dfm['arm_small']==1)
chongjang = (dfm['chongjang_big']==1)&(dfm['chongjang_small']==1)

shoulder_idx = dfm.loc[shoulder, :]['content'].index
chest_idx    = dfm.loc[chest,    :]['content'].index
arm_idx      = dfm.loc[arm,      :]['content'].index
chongjang_idx= dfm.loc[chongjang,:]['content'].index

전체 : 0
어깨 : 16
가슴 : 11
소매 : 12
총장 : 66


In [7]:
#11 데이터 전처리
#총장
dfm.loc[chongjang_idx[65],['chongjang_big','chongjang_small']] = 0,0 #19633
dfm.loc[chongjang_idx[64],'chongjang_small'] = 0 #19344
dfm.loc[chongjang_idx[63],['chongjang_big','chongjang_small']] = 0,0 #18989
dfm.loc[chongjang_idx[62],'chongjang_small'] = 0 #18619
dfm.loc[chongjang_idx[61],['chongjang_big','chongjang_small']] = 0,0 #18122
dfm.loc[chongjang_idx[60],['chongjang_big','chongjang_small']] = 0,0 #17490
dfm.loc[chongjang_idx[59],'chongjang_big'] = 0 #17058
dfm.loc[chongjang_idx[58],'chongjang_small'] = 0 #16693
dfm.loc[chongjang_idx[57],'chongjang_big'] = 0 #16063
dfm.loc[chongjang_idx[56],'chongjang_big'] = 0 #15776
dfm.loc[chongjang_idx[55],'chongjang_big'] = 0 #15761
dfm.loc[chongjang_idx[54],'chongjang_big'] = 0 #15583
dfm.loc[chongjang_idx[53],'chongjang_big'] = 0 #15282
dfm.loc[chongjang_idx[52],'chongjang_big'] = 0 #14787
dfm.loc[chongjang_idx[51],'chongjang_big'] = 0 #14326
dfm.loc[chongjang_idx[50],'chongjang_small'] = 0 #14259
dfm.loc[chongjang_idx[49],['chongjang_big','chongjang_small']] = 0,0 #14055
dfm.loc[chongjang_idx[48],['chongjang_big','chongjang_small']] = 0,0 #14051
dfm.loc[chongjang_idx[47],'chongjang_big'] = 0 #13987
dfm.loc[chongjang_idx[46],'chongjang_small'] = 0 #13844
dfm.loc[chongjang_idx[45],['chongjang_big','chongjang_small']] = 0,0 #13819
dfm.loc[chongjang_idx[44],['chongjang_big','chongjang_small']] = 0,0 #13096
dfm.loc[chongjang_idx[43],'chongjang_big'] = 0 #13089
dfm.loc[chongjang_idx[42],'chongjang_big'] = 0 #13055
dfm.loc[chongjang_idx[41],'chongjang_big'] = 0 #12782
dfm.loc[chongjang_idx[40],'chongjang_big'] = 0 #12457
dfm.loc[chongjang_idx[39],'chongjang_big'] = 0 #11711
dfm.loc[chongjang_idx[38],'chongjang_big'] = 0 #11694
dfm.loc[chongjang_idx[37],'chongjang_big'] = 0 #11682
dfm.loc[chongjang_idx[36],'chongjang_big'] = 0 #10991
dfm.loc[chongjang_idx[35],'chongjang_big'] = 0 #100801
dfm.loc[chongjang_idx[34],'chongjang_big'] = 0 #10787
dfm.loc[chongjang_idx[33],'chongjang_big'] = 0 #10650
dfm.loc[chongjang_idx[32],'chongjang_small'] = 0 #10591
dfm.loc[chongjang_idx[31],'chongjang_big'] = 0 #10533
dfm.loc[chongjang_idx[30],'chongjang_big'] = 0 #10516
dfm.loc[chongjang_idx[29],['chongjang_big','chongjang_small']] = 0,0 #10072
dfm.loc[chongjang_idx[28],'chongjang_big'] = 0 #8761
dfm.loc[chongjang_idx[27],'chongjang_big'] = 0 #8687
dfm.loc[chongjang_idx[26],'chongjang_big'] = 0 #8508
dfm.loc[chongjang_idx[25],'chongjang_small'] = 0 #8379
dfm.loc[chongjang_idx[24],'chongjang_small'] = 0 #8268
dfm.loc[chongjang_idx[23],'chongjang_big'] = 0 #7962
dfm.loc[chongjang_idx[22],'chongjang_big'] = 0 #7479
dfm.loc[chongjang_idx[21],'chongjang_big'] = 0 #7457
dfm.loc[chongjang_idx[20],'chongjang_big'] = 0 #7327
dfm.loc[chongjang_idx[19],'chongjang_big'] = 0 #7309
dfm.loc[chongjang_idx[18],'chongjang_big'] = 0 #6906
dfm.loc[chongjang_idx[17],['chongjang_big','chongjang_small']] = 0,0 #6649
dfm.loc[chongjang_idx[16],'chongjang_small'] = 0 #6452
dfm.loc[chongjang_idx[15],'chongjang_big'] = 0 #6404
dfm.loc[chongjang_idx[14],'chongjang_big'] = 0 #6251
dfm.loc[chongjang_idx[13],'chongjang_big'] = 0 #6201
dfm.loc[chongjang_idx[12],'chongjang_big'] = 0 #6200
dfm.loc[chongjang_idx[11],'chongjang_big'] = 0 #5654
dfm.loc[chongjang_idx[10],'chongjang_big'] = 0 #5640
dfm.loc[chongjang_idx[9],'chongjang_big'] = 0 #4885
dfm.loc[chongjang_idx[8],'chongjang_small'] = 0 #4458
dfm.loc[chongjang_idx[7],'chongjang_big'] = 0 #4162
dfm.loc[chongjang_idx[6],'chongjang_big'] = 0 #4100
dfm.loc[chongjang_idx[5],'chongjang_small'] = 0 #4003
dfm.loc[chongjang_idx[4],['chongjang_big','chongjang_small']] = 0,0 #3776
dfm.loc[chongjang_idx[3],'chongjang_big'] = 0 #3341
dfm.loc[chongjang_idx[2],'chongjang_big'] = 0 #2182
dfm.loc[chongjang_idx[1],'chongjang_big'] = 0 #1022
dfm.loc[chongjang_idx[0],'chongjang_big'] = 0 #1019

#소매
dfm.loc[arm_idx[0],['arm_big','arm_small']] = 0 #3776
dfm.loc[arm_idx[1],'arm_small'] = 0 #3973
dfm.loc[arm_idx[2],'arm_small'] = 0 #8379
dfm.loc[arm_idx[3],'arm_small'] = 0 #8647
dfm.loc[arm_idx[4],'arm_small'] = 0 #14334
dfm.loc[arm_idx[5],'arm_small'] = 0 #14787
dfm.loc[arm_idx[6],'arm_small'] = 0 #14822
dfm.loc[arm_idx[7],'arm_small'] = 0 #17135
dfm.loc[arm_idx[8],'arm_small'] = 0 #17267
dfm.loc[arm_idx[9],'arm_small'] = 0 #17490
dfm.loc[arm_idx[10],'arm_small'] = 0 #17582
dfm.loc[arm_idx[11],'arm_small'] = 0 #19344


#가슴
dfm.loc[chest_idx[0],'chest_small'] = 0 #2291
dfm.loc[chest_idx[1],'chest_small'] = 0 #3481
dfm.loc[chest_idx[2],['chest_small','chest_big']] = 0,0 #3891
dfm.loc[chest_idx[3],'chest_small'] = 0 #3973
dfm.loc[chest_idx[4],['chest_small','chest_big']] = 0,0 #5856
dfm.loc[chest_idx[5],['chest_small','chest_big']] = 0,0 #5858
dfm.loc[chest_idx[6],'chest_small'] = 0 #6651
dfm.loc[chest_idx[7],'chest_small'] = 0 #8647
dfm.loc[chest_idx[8],['chest_small','chest_big']] = 0 #14822
dfm.loc[chest_idx[9],'chest_big'] = 0 #15483
dfm.loc[chest_idx[10],['chest_big','chest_small']] = 0,0 #16142


#어깨
dfm.loc[shoulder_idx[0],'shoulder_small'] = 0 #2291
dfm.loc[shoulder_idx[1],'shoulder_small'] = 0 #3481
dfm.loc[shoulder_idx[2],'shoulder_small'] = 0 #3891
dfm.loc[shoulder_idx[3],'shoulder_big']   = 0 #3905
dfm.loc[shoulder_idx[4],'shoulder_small'] = 0 #3973
dfm.loc[shoulder_idx[5],['shoulder_big','shoulder_small']] = 0,0 #5856
dfm.loc[shoulder_idx[6],['shoulder_big','shoulder_small']] = 0,0 #5858
dfm.loc[shoulder_idx[7],'shoulder_small'] = 0 #6651
dfm.loc[shoulder_idx[8],'shoulder_small'] = 0 #8647
dfm.loc[shoulder_idx[9],'shoulder_big'] = 0 #11917
dfm.loc[shoulder_idx[10],['shoulder_big','shoulder_small']] = 0,0 #14822
dfm.loc[shoulder_idx[11],'shoulder_big'] = 0 #15483
dfm.loc[shoulder_idx[12],['shoulder_big','shoulder_small']] = 0,0 #16054
dfm.loc[shoulder_idx[13],['shoulder_big','shoulder_small']] = 0,0 #16142
dfm.loc[shoulder_idx[14],['shoulder_big','shoulder_small']] = 0,0 #17721
dfm.loc[shoulder_idx[15],'shoulder_small'] = 0 #18183

In [8]:
print(dfm[(dfm['total_big']==1)&(dfm['total_small']==1)].shape[0])
print(dfm[(dfm['shoulder_big']==1)&(dfm['shoulder_small']==1)].shape[0])
print(dfm[(dfm['arm_big']==1)&(dfm['arm_small']==1)].shape[0])
print(dfm[(dfm['chest_big']==1)&(dfm['chest_small']==1)].shape[0])
print(dfm[(dfm['chongjang_big']==1)&(dfm['chongjang_small']==1)].shape[0])


0
0
0
0
0


In [10]:
dfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19853 entries, 0 to 19852
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            19853 non-null  int64  
 1   height            19853 non-null  float64
 2   weight            19853 non-null  float64
 3   size              19853 non-null  object 
 4   content           19853 non-null  object 
 5   총장                19853 non-null  float64
 6   어깨너비              19035 non-null  float64
 7   가슴단면              19853 non-null  float64
 8   소매길이              19852 non-null  float64
 9   total_big         19853 non-null  int64  
 10  total_small       19853 non-null  int64  
 11  chongjang_big     19853 non-null  int64  
 12  chongjang_small   19853 non-null  int64  
 13  arm_big           19853 non-null  int64  
 14  arm_small         19853 non-null  int64  
 15  chest_big         19853 non-null  int64  
 16  chest_small       19853 non-null  int64 

In [11]:
dfm.to_pickle('data/Modeling_DF_230116.pickle')

In [219]:
X_chongjang = dfm.iloc[:, [0,1,2,10,11,19]]; y_chongjang = dfm.iloc[:,4]

#데이터분할
X_train_chongjang, X_test_chongjang, y_train_chongjang, y_test_chongjang = train_test_split(X_chongjang, y_chongjang, test_size=0.2, shuffle=True, random_state=230115)

#학습데이터, 검증데이터(x,y 모두 포함)
train_chongjang = pd.concat([X_train_chongjang, y_train_chongjang], axis=1)
test_chongjang  = pd.concat([X_test_chongjang,  y_test_chongjang] , axis=1)

#학습데이터, 검증데이터 라벨 분리
X_train_chongjang_big = train_chongjang.loc[train_chongjang['chongjang_big']==1,:][['gender','height','weight']]
y_train_chongjang_big = train_chongjang.loc[train_chongjang['chongjang_big']==1,:]['총장']
X_test_chongjang_big  = test_chongjang.loc[test_chongjang['chongjang_big']==1  ,:][['gender','height','weight']]
y_test_chongjang_big  = test_chongjang.loc[test_chongjang['chongjang_big']==1  ,:]['총장']

X_cols = X_train_chongjang_big.columns.tolist()

print('총장이 큰 경우, 학습데이터 개수 : {}, 검증데이터 개수 : {}'.format(len(y_train_chongjang_big), len(y_test_chongjang_big)))

총장이 큰 경우, 학습데이터 개수 : 532, 검증데이터 개수 : 125


랜포

In [180]:
#최적의 하이퍼파라미터 탐색, GridSearchCV
from sklearn.model_selection import GridSearchCV

param_grid = [
        {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
        {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
    ]
rfr_chongjang_big = RandomForestRegressor()
 
grid_search = GridSearchCV(rfr_chongjang_big, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
 
grid_search.fit(X_train_chongjang_big, y_train_chongjang_big)
grid_search.best_params_

{'max_features': 2, 'n_estimators': 30}

In [187]:
final_model = grid_search.best_estimator_

pred = final_model.predict(X_test_chongjang_big) #예측값
true = y_test_chongjang_big #실제값
res = pred-true #잔차

In [208]:
plus = pd.DataFrame({'true':y_test_chongjang_big.tolist(), 'pred':pred.tolist(), 'res':res.tolist()} )
X_test_chongjang_big.reset_index(drop=True, inplace=True)
RF_chongjang_big = pd.concat([X_test_chongjang_big, plus], axis=1)
RF_chongjang_big

,gender,height,weight,true,pred,res
0,0,169.0,60.0,76.0,66.135501,-9.864499
1,0,157.0,46.0,71.0,66.107778,-4.892222
2,0,163.0,51.0,76.0,63.496230,-12.503770
3,0,165.0,62.0,71.0,70.711111,-0.288889
4,0,176.0,75.0,74.0,72.819444,-1.180556
...,...,...,...,...,...,...
120,0,161.0,62.0,66.0,65.533333,-0.466667
121,1,184.0,93.0,73.0,73.066667,0.066667
122,0,152.0,43.0,64.0,61.444444,-2.555556
123,1,173.0,68.0,68.0,68.178333,0.178333


다중선형회귀

In [212]:
from sklearn.linear_model import LinearRegression

mlr_chongjang_big = LinearRegression()
mlr_chongjang_big.fit(X_train_chongjang_big, y_train_chongjang_big)

LinearRegression()

In [213]:
pred2 = mlr_chongjang_big.predict(X_test_chongjang_big)
true2 = y_test_chongjang_big
res2  = pred2 - true2

In [214]:
plus2 = pd.DataFrame({'true':y_test_chongjang_big.tolist(), 'pred':pred2.tolist(), 'res':res2.tolist()} )
X_test_chongjang_big.reset_index(drop=True, inplace=True)
MLR_chongjang_big = pd.concat([X_test_chongjang_big, plus2], axis=1)
MLR_chongjang_big

,gender,height,weight,true,pred,res
0,0,169.0,60.0,76.0,69.992219,-6.007781
1,0,157.0,46.0,71.0,67.073089,-3.926911
2,0,163.0,51.0,76.0,68.413453,-7.586547
3,0,165.0,62.0,71.0,69.416512,-1.583488
4,0,176.0,75.0,74.0,72.102315,-1.897685
...,...,...,...,...,...,...
120,0,161.0,62.0,66.0,68.721604,2.721604
121,1,184.0,93.0,73.0,74.126548,1.126548
122,0,152.0,43.0,64.0,66.025652,2.025652
123,1,173.0,68.0,68.0,70.725539,2.725539


- [밤에한헛생각들...]
- 일단 실제 구매보다 총장이 더 크게 예측된게 56/125개 혹은 60/125개 절반넘네.....
- 근데 총장이 길다고한 사람들은 예측이 작게 될수록 좋은 것(잔차가 음의 방향으로 클때) 아닌가?
- 기존에는 abs(예측된 수치➖사용자가 실제로 산 제품의 수치)가 최소일 때는 잘못 구매한 사이즈의 총장 수치와 비교이므로
- (길다고한 경우) 고객이 구매한 사이즈보다 한 사이즈 작은 총장 수치와 예측값의 차이를 봐야하는 것 아닌가....
- 근데 이렇게 되면 상품별로 사이즈별 치수 폭이 다르니까 ....하...